## **Artificial Neural Networks**

Import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

**MNIST dataset**

*   Database of handwritten digits
*   Training set: 60,000 samples
*   Testing set: 10,000 samples
*   10 classes of 28 x 28 images

![alt text](https://drive.google.com/uc?id=1hZrTmIUAYpfwWJp_wRYs3I6VwIlhVbnZ)

Load training and test dataset

In [ ]:
BATCH_SIZE = 32

## transformations
transform = transforms.Compose(
    [transforms.ToTensor()])

## download and load training dataset
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
mnist_trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=1)

## download and load testing dataset
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
mnist_testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=1)

In [ ]:
print(mnist_trainset)
print(mnist_testset)

Visualize MNIST dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## get some random training images
mnist_dataiter = iter(mnist_trainloader)
images, labels = mnist_dataiter.next()

## show images
images_grid = torchvision.utils.make_grid(images)
plt.imshow(np.transpose(images_grid.numpy(), (1, 2, 0)))
plt.axis('off')
plt.show()

In [ ]:
for images, labels in mnist_trainloader:
    print("Train - image batch dimensions:", images.shape)
    print("Train - image label dimensions:", labels.shape)
    break

for images, labels in mnist_testloader:
    print("Test - image batch dimensions:", images.shape)
    print("Test - image label dimensions:", labels.shape)
    break

Create a simple model

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()

        self.convlayer = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.fclayer1 = nn.Linear(26 * 26 * 32, 128)
        self.fclayer2 = nn.Linear(128, 10)

    def forward(self, x):
        # input: 32x1x28x28 -> output: 32x32x26x26
        x = self.convlayer(x)
        x = F.relu(x)

        # input: 32x32x26x26 -> output: 32x(32*26*26)
        x = x.flatten(start_dim = 1)

        # input: 32x(32*26*26) -> output: 32x128
        x = self.fclayer1(x)
        x = F.relu(x)

        # input: 32x128 -> output: 32x10 (32 images, 10 classes)
        logits = self.fclayer2(x)
        out = F.softmax(logits, dim=1)
        return out

Test if the model works well

In [ ]:
model = SimpleModel()
for images, labels in mnist_trainloader:
    print("batch size:", images.shape)
    out = model(images)
    print(out.shape)
    break

Let's train the model

In [ ]:
## compute stat
def compute_stat(outputs, labels, stats):
    _, preds = torch.max(outputs, 1)
    stats['n_corr'] += torch.sum(preds == labels).item()
    stats['n'] += torch.numel(labels)

    return stats

def train_model(model, criterion, optimizer, trainloader, device, num_epochs=25):
  epoch_stat = {'n_corr': 0, 'n': 0}

  for epoch in range(num_epochs):
      train_running_loss = 0.0
      train_acc = 0.0

      model.train()  # Set model to training mode

      ## training step
      for i, (images, labels) in enumerate(trainloader):
          
          images = images.to(device)
          labels = labels.to(device)

          ## forward + backprop + loss
          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()

          ## update model params
          optimizer.step()

          train_running_loss += loss.detach().item()
          epoch_stat = compute_stat(outputs, labels, epoch_stat)
      
      model.eval()
      print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
            %(epoch, train_running_loss / i, epoch_stat['n_corr'] / epoch_stat['n'] * 100)) 
      
  return model

In [ ]:
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SimpleModel()
model = model.to(device)

## loss function
criterion = nn.CrossEntropyLoss()
## optimization
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model = train_model(model, criterion, optimizer, mnist_trainloader, device, num_epochs=5)

Test on the test dataset

In [ ]:
model.eval() # Set model to evaluate mode

test_stat = {'n_corr': 0, 'n': 0}
for i, (images, labels) in enumerate(mnist_testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    test_stat = compute_stat(outputs, labels, test_stat)
        
print('Test Accuracy: %.2f'%( test_stat['n_corr'] / test_stat['n'] * 100))

## **Utilize the existing models & transfer learning**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torchsummary import summary

**CIFAR10 dataset**

*   Database of natural images
*   Training set: 50,000 samples
*   Testing set: 10,000 samples
*   10 classes of 32 x 32 images

![alt text](https://drive.google.com/uc?id=1qQFYheGrOa3Rj3nlcf2UXo26yTSwu6Ak)

Load CIFAR10 dataset

In [ ]:
BATCH_SIZE = 64

## transformations
transform = transforms.Compose(
    [transforms.ToTensor()])

## download and load training dataset
cifar_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
cifar_trainloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=1)

## download and load testing dataset
cifar_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
cifar_testloader = torch.utils.data.DataLoader(cifar_testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## get some random training images
dataiter = iter(cifar_trainloader)
images, labels = dataiter.next()

## show images
images_grid = torchvision.utils.make_grid(images)
plt.imshow(np.transpose(images_grid.numpy(), (1, 2, 0)))
plt.axis('off')
plt.show()

In [ ]:
for images, labels in cifar_trainloader:
    print("Train - image batch dimensions:", images.shape)
    print("Train - image label dimensions:", labels.shape)
    break

for images, labels in cifar_testloader:
    print("Test - image batch dimensions:", images.shape)
    print("Test - image label dimensions:", labels.shape)
    break

Load the existing model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load the existing model (Resnet18) 
# pretrained model: use weights from ImageNet dataset
model_ft = models.resnet18(pretrained=True)
model_ft = model_ft.to(device)

In [ ]:
print(model_ft)

In [ ]:
summary(model_ft, input_size=(3,32,32)) 

Check the last fully-connected (fc) layer

In [ ]:
print(model_ft.fc)

In [ ]:
fc_num_in_fts = model_ft.fc.in_features
fc_num_out_fts = model_ft.fc.out_features

print('LAST - fc: ', fc_num_in_fts, fc_num_out_fts)

Change 'fc' to fit to CIFAR19 dataset

In [ ]:
model_ft.fc = nn.Linear(fc_num_in_fts, 10)  # 10 classes

In [ ]:
model_ft = model_ft.to(device)
summary(model_ft, input_size=(3,32,32))

Let's train the model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## loss function
criterion_ft = nn.CrossEntropyLoss()
## optimization
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-2, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-4)

model_ft = train_model(model_ft, criterion_ft, optimizer_ft, cifar_trainloader, device, num_epochs=5)

Test on the test dataset

In [ ]:
test_stat = {'n_corr': 0, 'n': 0}
for i, (images, labels) in enumerate(cifar_testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model_ft(images)
    test_stat = compute_stat(outputs, labels, test_stat)
        
print('Test Accuracy: %.2f'%( test_stat['n_corr'] / test_stat['n'] * 100))

Change 'conv1' and 'fc' to fit to MNIST dataset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load the existing model (Resnet18) 
# pretrained model: use weights from ImageNet dataset
model_ft2 = models.resnet18(pretrained=True)
model_ft2 = model_ft2.to(device)

In [ ]:
print(model_ft2.conv1)

In [ ]:
conv1_num_in_fts2 = model_ft2.conv1.in_channels
conv1_num_out_fts2 = model_ft2.conv1.out_channels

print('FIRST - conv1 : ', conv1_num_in_fts2, conv1_num_out_fts2)

fc_num_in_fts2 = model_ft2.fc.in_features
fc_num_out_fts2 = model_ft2.fc.out_features

print('LAST - fc: ', fc_num_in_fts2, fc_num_out_fts2)

In [ ]:
model_ft2.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=1, padding=3, bias=False)

model_ft2.fc = nn.Linear(fc_num_in_fts2, 10)  # 10 classes

In [ ]:
model_ft2 = model_ft2.to(device)
summary(model_ft2, input_size=(1,28,28))

In [ ]:
## loss function
criterion_ft2 = nn.CrossEntropyLoss()
## optimization
#optimizer_ft2 = optim.SGD(model_ft2.parameters(), lr=1e-2, momentum=0.9)
optimizer_ft2 = optim.Adam(model_ft2.parameters(), lr=1e-3)

model_ft2 = train_model(model_ft2, criterion_ft2, optimizer_ft2, mnist_trainloader, device, num_epochs=5)


In [ ]:
test_stat = {'n_corr': 0, 'n': 0}
for i, (images, labels) in enumerate(mnist_testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model_ft2(images)
    test_stat = compute_stat(outputs, labels, test_stat)
        
print('Test Accuracy: %.2f'%( test_stat['n_corr'] / test_stat['n'] * 100))